In [1]:
import pickle
import time
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
from collections import Counter
import string
from sklearn import (
    linear_model,
    naive_bayes,
    svm,
    metrics,
    tree,
    neighbors,
    decomposition,
)
import ast
import importlib
import numpy as np
from scipy.spatial import distance
import random
import nltk
import pandas as pd


In [2]:
import sys, os

sys.path.insert(0, os.path.abspath("../cse258_hw/"))



In [3]:
import as2_analysis_utils as as2_analysis

importlib.reload(as2_analysis)



<module 'as2_analysis_utils' from '/Users/kwkarlwang/Desktop/Master/cse258/assignment2/as2_analysis_utils.py'>

In [4]:
data_all = list(as2_analysis.parseData_line("renttherunway_final_data.json"))



In [5]:
def extract_features(data_all, key):
    """
    return data of particualr key
    """
    return [d[key] for d in data_all]



In [6]:
rating_all = extract_features(data_all, "rating")



In [7]:
null_count = [i == "null" for i in rating_all]



In [8]:
sum(null_count) / len(null_count)



0.0004258766827322586

In [9]:
fit_feedbadk = extract_features(data_all, "fit")



In [10]:
fit_feedbadk_freq = Counter(fit_feedbadk)



In [11]:
fit_feedbadk_prct = {key: val / len(data_all) for key, val in fit_feedbadk_freq.items()}



In [12]:
fit_feedbadk_prct


{'fit': 0.7377949975070633,
 'small': 0.13388628053847432,
 'large': 0.12831872195446237}

 # Count word frequency

In [13]:
# top words in fit reviews
fit_reviews_all = [
    d["review_text"] + " " + d["review_summary"] for d in data_all if d["fit"] == "fit"
]
small_reviews_all = [
    d["review_text"] + " " + d["review_summary"]
    for d in data_all
    if d["fit"] == "small"
]
large_reviews_all = [
    d["review_text"] + " " + d["review_summary"]
    for d in data_all
    if d["fit"] == "large"
]



In [14]:
def tokenize_data_sets(dataset, n=1):
    """
    Each item of dataset is a str
    """
    return [as2_analysis.tokenize_paragraph(d, n=n) for d in dataset]



In [15]:
# tokenize dataset
fit_tokenized_all = tokenize_data_sets(fit_reviews_all)
small_tokenized_all = tokenize_data_sets(small_reviews_all)
large_tokenized_all = tokenize_data_sets(large_reviews_all)



In [16]:
fit_word_count = as2_analysis.count_word_freq(fit_tokenized_all)
small_word_count = as2_analysis.count_word_freq(small_tokenized_all)
large_word_count = as2_analysis.count_word_freq(large_tokenized_all)


 # Get most frequent adjective

In [17]:
def top_n_adj(word_count_dict):
    ## get the most common ajectives
    adj_count = {
        key: word_count_dict[key]
        for key in word_count_dict
        if nltk.pos_tag([key])[0][1][0] == "J"
    }

    # sort them into pairs
    adj_count_pair = [(count, key) for key, count in adj_count.items()]

    # sort
    adj_count_pair.sort(reverse=True)

    return adj_count_pair



In [18]:
start = time.time()
top_small_adj = top_n_adj(small_word_count)


top_big_adj = top_n_adj(large_word_count)


## get the most common ajectives
adj_fit_count = {
    key: fit_word_count[key]
    for key in fit_word_count
    if nltk.pos_tag([key])[0][1][0] == "J"
}

end = time.time()
print(f"time consume: {end-start}s")

time consume: 11.68418002128601s


In [19]:
adj_fit_count_pair = [(count, key) for key, count in adj_fit_count.items()]



In [20]:
adj_fit_count_pair.sort(reverse=True)



In [21]:
adj_fit_count_pair[:10]



[(60010, 'great'),
 (48689, 'comfortable'),
 (39921, 'little'),
 (22857, 'many'),
 (18916, 'true'),
 (17789, 'short'),
 (15455, 'black'),
 (13260, 'gorgeous'),
 (12799, 'much'),
 (11649, 'nice')]

In [22]:
top_small_adj[:10]



[(11137, 'small'),
 (9126, 'great'),
 (8424, 'little'),
 (5654, 'comfortable'),
 (4705, 'short'),
 (3386, 'many'),
 (2854, 'gorgeous'),
 (2675, 'overall'),
 (2623, 'much'),
 (2620, 'other')]

In [23]:
top_big_adj[:10]



[(9276, 'great'),
 (8972, 'little'),
 (8489, 'large'),
 (7584, 'comfortable'),
 (7517, 'big'),
 (3201, 'small'),
 (3011, 'many'),
 (2654, 'much'),
 (2602, 'overall'),
 (2406, 'smaller')]

 Try bigram and trigram

In [24]:
def top_n_adj_n_gram(word_count_dict, adj_set=None):

    # if set of adj is given
    if adj_set:
        adj_count = {
            key: word_count_dict[key]
            for key in word_count_dict
            # if either of the word is an adj, preserve
            if key in adj_set
        }
    else:
        ## get the most common ajectives, might be slow
        adj_count = {
            key: word_count_dict[key]
            for key in word_count_dict
            # if either of the word is an adj, preserve
            if any(
                wordType[0] == "J"
                for _, wordType in nltk.pos_tag(key if type(key) == tuple else (key,))
            )
        }

    # sort them into pairs
    adj_count_pair = list(adj_count.items())

    # # sort
    adj_count_pair.sort(reverse=True, key=lambda x: x[1])

    return adj_count_pair



In [25]:
def top_adj_pipeline(fit_reviews_all, small_reviews_all, large_reviews_all, n=2):

    fit_tokenized_all = tokenize_data_sets(fit_reviews_all, n)
    small_tokenized_all = tokenize_data_sets(small_reviews_all, n)
    large_tokenized_all = tokenize_data_sets(large_reviews_all, n)

    fit_word_count = as2_analysis.count_word_freq(fit_tokenized_all)
    small_word_count = as2_analysis.count_word_freq(small_tokenized_all)
    large_word_count = as2_analysis.count_word_freq(large_tokenized_all)

    # all adj for fit, small, and large
    fit_adj, small_adj, large_adj = None, None, None

    # load if the pickle exists
    if os.path.exists(f"{n}-grams_adj"):
        with open(f"{n}-grams_adj", "rb") as f:
            fit_adj, small_adj, large_adj = pickle.load(f)

    start = time.time()
    top_fit_grams = top_n_adj_n_gram(fit_word_count, fit_adj)
    top_small_grams = top_n_adj_n_gram(small_word_count, small_adj)
    top_large_grams = top_n_adj_n_gram(large_word_count, large_adj)
    end = time.time()
    print(f"time consume: {end-start}s")
    return top_fit_grams, top_small_grams, top_large_grams



In [26]:
def write_adj_to_file(n):
    # get all the adj from the entire data
    top_fit_grams, top_small_grams, top_large_grams = top_adj_pipeline(
        fit_reviews_all, small_reviews_all, large_reviews_all, n
    )
    # make into sets
    a, b, c = (
        {word for word, _ in top_fit_grams},
        {word for word, _ in top_small_grams},
        {word for word, _ in top_large_grams},
    )
    # write to file
    with open(f"./{n}-grams_adj", "wb") as f:
        pickle.dump((a, b, c), f)



In [27]:
top_fit_grams, top_small_grams, top_large_grams = top_adj_pipeline(
    fit_reviews_all, small_reviews_all, large_reviews_all, 2
)

time consume: 0.3970160484313965s


In [28]:

top_fit_grams[:20]

[(('a', 'little'), 37021),
 (('was', 'perfect'), 21087),
 (('many', 'compliments'), 18749),
 (('true', 'to'), 17248),
 (('so', 'i'), 16506),
 (('so', 'many'), 14559),
 (('comfortable', 'and'), 12829),
 (('very', 'comfortable'), 11175),
 (('the', 'top'), 9822),
 (('a', 'great'), 9118),
 (('great', 'dress'), 8835),
 (('able', 'to'), 7224),
 (('easy', 'to'), 7027),
 (('was', 'great'), 7012),
 (('great', 'for'), 6568),
 (('beautiful', 'dress'), 6409),
 (('the', 'only'), 6378),
 (('and', 'comfortable'), 5396),
 (('black', 'tie'), 5095),
 (('was', 'comfortable'), 5011)]

In [29]:
top_small_grams[:20]

[(('a', 'little'), 7847),
 (('so', 'i'), 4534),
 (('many', 'compliments'), 2749),
 (('was', 'perfect'), 2652),
 (('runs', 'small'), 2557),
 (('the', 'top'), 2184),
 (('so', 'many'), 2041),
 (('able', 'to'), 1758),
 (('beautiful', 'dress'), 1729),
 (('small', 'i'), 1694),
 (('great', 'dress'), 1598),
 (('too', 'tight'), 1484),
 (('a', 'great'), 1465),
 (('too', 'short'), 1205),
 (('comfortable', 'and'), 1201),
 (('too', 'small'), 1119),
 (('normally', 'wear'), 1107),
 (('very', 'comfortable'), 1092),
 (('small', 'and'), 1053),
 (('little', 'small'), 1046)]

In [30]:
top_large_grams[:20]



[(('a', 'little'), 8558),
 (('so', 'i'), 3870),
 (('was', 'perfect'), 2582),
 (('the', 'top'), 2401),
 (('many', 'compliments'), 2366),
 (('too', 'big'), 2096),
 (('comfortable', 'and'), 2026),
 (('runs', 'large'), 1842),
 (('so', 'many'), 1829),
 (('very', 'comfortable'), 1735),
 (('little', 'big'), 1542),
 (('a', 'great'), 1539),
 (('great', 'dress'), 1404),
 (('able', 'to'), 1390),
 (('large', 'i'), 1381),
 (('beautiful', 'dress'), 1189),
 (('little', 'large'), 1174),
 (('great', 'for'), 1156),
 (('easy', 'to'), 1057),
 (('a', 'small'), 976)]

 # Feature Engineering

In [31]:
fit_data = [d for d in data_all if d["fit"] == "fit"]
small_data = [d for d in data_all if d["fit"] == "small"]
large_data = [d for d in data_all if d["fit"] == "large"]
data_balanced = random.sample(fit_data, k=len(large_data)) + small_data + large_data

In [32]:
random.shuffle(data_balanced)

In [33]:
data_size = len(data_balanced)
valid_percent = 0.2
test_percent = 0.2



In [34]:

data_train = data_balanced[: int(data_size * (1 - valid_percent - test_percent))]
data_valid = data_balanced[
    int(data_size * (1 - valid_percent - test_percent)) : int(
        data_size * (1 - test_percent)
    )
]
data_test = data_balanced[int(data_size * (1 - test_percent)) :]


In [35]:
def extract_review(data):
    fit_reviews = [
        d["review_text"] + " " + d["review_summary"] for d in data if d["fit"] == "fit"
    ]
    small_reviews = [
        d["review_text"] + " " + d["review_summary"]
        for d in data
        if d["fit"] == "small"
    ]
    large_reviews = [
        d["review_text"] + " " + d["review_summary"]
        for d in data
        if d["fit"] == "large"
    ]
    return fit_reviews, small_reviews, large_reviews



 Try n-gram BoW logistic regression/navie bayes

In [36]:
fit_reviews_train, small_reviews_train, large_reviews_train = extract_review(data_train)


In [37]:
ns = [1, 2, 3, 4, 5]
thresholds = [100, 500, 1000, 2000, 3000]
top_word_set = {"small", "large", "big"}
for n, threshold in zip(ns, thresholds):
    top_fit_grams, top_small_grams, top_large_grams = top_adj_pipeline(
        fit_reviews_train, small_reviews_train, large_reviews_train, n
    )

    a, b, c = (
        {word for word, _ in top_fit_grams[:threshold]},
        {word for word, _ in top_small_grams[:threshold]},
        ({word for word, _ in top_large_grams[:threshold]}),
    )
    top_gram = (a | b | c) - (a & b & c)
    print(f"unique top {n}-grams: {len(top_gram)}")
    top_word_set |= top_gram

print(f"unique top grams: {len(top_word_set)}")

wordId = dict(zip(top_word_set, range(len(top_word_set))))

time consume: 0.005340099334716797s
unique top 1-grams: 18
time consume: 0.14866280555725098s
unique top 2-grams: 337
time consume: 0.6317377090454102s
unique top 3-grams: 1037
time consume: 1.2195277214050293s
unique top 4-grams: 2758
time consume: 1.7382757663726807s
unique top 5-grams: 5131
unique top grams: 9284


In [38]:
def feature(d, ns=[1]):
    feat = [0] * len(top_word_set)
    review = d["review_text"] + " " + d["review_summary"]
    p_list = []
    for n in ns:
        p_list.extend(as2_analysis.tokenize_paragraph(review, n=n))
    for word in p_list:
        if word not in top_word_set:
            continue
        # use BoW for now
        feat[wordId[word]] += 1
    return feat



In [39]:
def encode_output(data):
    return [mapping[d["fit"]] for d in data]



In [40]:
mapping = {"fit": 0, "small": 1, "large": 2}
X_train = [feature(d, ns) for d in data_train]
y_train = encode_output(data_train)
X_valid = [feature(d, ns) for d in data_valid]
y_valid = encode_output(data_valid)



 # Modeling

 Try the following numerous text classfication algorithm:
 
    - Navie Bayes
    - SVM
    - Logistic Regression
    - Decision Tree
    - KNN

In [41]:
models_data = {}

 ## Navie Bayes

In [42]:
model = naive_bayes.MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
print("Navie Bayes classifcation report\n")
print(metrics.classification_report(y_valid, y_pred))
models_data["naive bayes"] = metrics.classification_report(
    y_valid, y_pred, output_dict=1
)


Navie Bayes classifcation report

              precision    recall  f1-score   support

           0       0.56      0.59      0.58      4975
           1       0.61      0.69      0.65      5168
           2       0.74      0.61      0.67      4896

    accuracy                           0.63     15039
   macro avg       0.64      0.63      0.63     15039
weighted avg       0.64      0.63      0.63     15039



## SVM

Try the following kernel:

    - linear
    - rbf
    - polynomial

In [43]:
regs = [0.01, 1, 10, 100]
degs = [2, 3, 4]
models = {f"kernel=linear, C={reg}": svm.LinearSVC(C=reg) for reg in regs}
# models.update(
#     {f"kernel=rbf, C={reg}": svm.SVC(kernel="rbf", C=reg, gamma="auto") for reg in regs}
# )
# models.update(
#     {
#         f"kernel=poly, C={reg}, deg={deg}": svm.SVC(kernel="poly", C=reg, gamma="auto")
#         for reg in regs
#         for deg in degs
#     }
# )
for desc, model in models.items():
    if f"svm, {desc}" in models_data:
        continue
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    print(f"SVM {desc} classifcation report\n")
    print(metrics.classification_report(y_valid, y_pred))
    models_data[f"svm, {desc}"] = metrics.classification_report(
        y_valid, y_pred, output_dict=1
    )



SVM kernel=linear, C=0.01 classifcation report

              precision    recall  f1-score   support

           0       0.54      0.76      0.63      4975
           1       0.76      0.61      0.68      5168
           2       0.76      0.60      0.67      4896

    accuracy                           0.66     15039
   macro avg       0.69      0.66      0.66     15039
weighted avg       0.69      0.66      0.66     15039

SVM kernel=linear, C=1 classifcation report

              precision    recall  f1-score   support

           0       0.53      0.66      0.59      4975
           1       0.70      0.61      0.66      5168
           2       0.68      0.61      0.64      4896

    accuracy                           0.63     15039
   macro avg       0.64      0.63      0.63     15039
weighted avg       0.64      0.63      0.63     15039

SVM kernel=linear, C=10 classifcation report

              precision    recall  f1-score   support

           0       0.53      0.64      0.58 

 ## Logistic Regression

In [44]:

regs = [0.01, 1, 10, 100]
weights = [None]
models = {
    f"C={reg}, weights={weight}": linear_model.LogisticRegression(
        C=reg, class_weight=weight
    )
    for reg in regs
    for weight in weights
}


for desc, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    print(f"Logistic Regression {desc} classifcation report\n")
    print(metrics.classification_report(y_valid, y_pred))
    models_data[f"logistic regression, {desc}"] = metrics.classification_report(
        y_valid, y_pred, output_dict=1
    )


Logistic Regression C=0.01, weights=None classifcation report

              precision    recall  f1-score   support

           0       0.51      0.80      0.63      4975
           1       0.77      0.56      0.64      5168
           2       0.77      0.55      0.64      4896

    accuracy                           0.64     15039
   macro avg       0.68      0.64      0.64     15039
weighted avg       0.68      0.64      0.64     15039

Logistic Regression C=1, weights=None classifcation report

              precision    recall  f1-score   support

           0       0.54      0.69      0.61      4975
           1       0.73      0.62      0.67      5168
           2       0.71      0.63      0.66      4896

    accuracy                           0.65     15039
   macro avg       0.66      0.65      0.65     15039
weighted avg       0.66      0.65      0.65     15039

Logistic Regression C=10, weights=None classifcation report

              precision    recall  f1-score   support


 ## Decision Tree

In [45]:
# depths = [50, 100, 200]
# models = {
#     f"max_depth={depth}": tree.DecisionTreeClassifier(max_depth=depth)
#     for depth in depths
# }


# for desc, model in models.items():
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_valid)
#     print(f"Decision Tree {desc} classifcation report\n")
#     print(metrics.classification_report(y_valid, y_pred))
#     models_data[f"decision tree, {desc}"] = metrics.classification_report(
#         y_valid, y_pred, output_dict=1
#     )



 ## K-nearest neighbor

In [46]:
# neighbor_nums = [1, 5, 10, 20, 50, 100]
# models = {
#     f"n-neighbor={neighbor_num}": neighbors.KNeighborsClassifier(
#         n_neighbors=neighbor_num
#     )
#     for neighbor_num in neighbor_nums
# }
# for desc, model in models.items():
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_valid)
#     print(f"KNN {desc} classifcation report\n")
#     print(metrics.classification_report(y_valid, y_pred))
#     models_data[f"knn, {desc}"] = metrics.classification_report(
#         y_valid, y_pred, output_dict=1
#     )

